In [ ]:
pip install openai

In [ ]:
pip install random

In [ ]:
pip install pandas

In [ ]:
pip install xlrd

In [ ]:
pip install xlwt

In [ ]:
pip install xlutils

In [ ]:
pip install faker

In [ ]:
!pip install --upgrade faker

In [ ]:
!pip install random

In [ ]:
import openai
import pandas as pd
import random
from openpyxl import Workbook
from openpyxl.utils import get_column_letter
from faker import Faker
from datetime import datetime, timedelta


# Set up OpenAI API credentials
openai.api_key = "YOUR_API_KEY"

# Define the keywords to detect
keywords = ["Necklace", "Bracelet", "Earring", "Ring", "Gold", "Diamond", "Personalized", "Name", "Custom", "necklace", "bracelet", "earring", "ring", "gold", "diamond", "personalized", "name"]

# Define the emojis for good and bad ratings
good_emojis = ["😍", "🤩", "👏", "👌", "👍", "🙌", "🌟"]
bad_emojis = ["😞", "😕", "😟", "😒", "🙁", "💔"]

# Load the input Excel file
input_file_path = "input.xlsx"
df = pd.read_excel(input_file_path)

# Initialize a new Excel file for the output
output_file_path = "output.xlsx"
workbook = Workbook()
worksheet = workbook.active
worksheet.title = "Sheet1"
worksheet.append(["text", "style", "emoji", "rating", "author", "body", "created_at"])

# Initialize Faker with 'female' gender
fake = Faker('en_US')

# Loop through the input data and detect keywords, paraphrase text using ChatGPT, and add a random emoji and name based on the rating
for i in range(len(df)):
    text = df.iloc[i]["text"]
    rating = df.iloc[i]["rating"]
    detected_keywords = [word for word in keywords if word in text]
    style = ", ".join(detected_keywords)
    if not style:
        style = "None"
    response = openai.Completion.create(
        engine="text-davinci-002",
        prompt=f"Paraphrase the following text: {text}",
        temperature=0.7,
        max_tokens=1024,
        n=1,
        stop=None,
        timeout=15,
    )
    body = response.choices[0].text.strip()
    
    # Add a random emoji and name based on the rating
    if rating <= 3:
        emoji = random.choice(bad_emojis)
        author = fake.first_name_female() + " " + fake.last_name_female()

    else:
        emoji = random.choice(good_emojis)
        author = fake.first_name_female() + " " + fake.last_name_female()
    
    # Generate a random past date within the last 30 days
    past_date = datetime.now() - timedelta(days=random.randint(1, 30))

    # Set the year to the current year
    created_at = past_date.replace(year=datetime.now().year).strftime("%Y-%m-%d")

    # Write the results to the output Excel file
    row = [text, style, emoji, rating, author, body, created_at]
    worksheet.append(row)


# Save the output Excel file
for col_num in range(len(row)):
    col_letter = get_column_letter(col_num + 1)
    column_dimensions = worksheet.column_dimensions[col_letter]
    column_dimensions.width = 20
workbook.save(output_file_path)
